In [3]:
import os
import sys
import glob

import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.backend.tensorflow_backend import set_session

import matplotlib.pyplot as plt

import nibabel as nib

Using TensorFlow backend.


In [4]:
def plot(img):
    plt.figure()
    plt.imshow(img[:, 40, :], cmap='gray')
    
def load_nii(path):
    nii = nib.load(path)
    vol = nii.get_data().astype(np.float32)
    return np.squeeze(vol)

In [5]:
run_dirs = glob.glob('../runs/*/predict')
run_dirs = sorted(run_dirs)

print('found {} runs'.format(len(run_dirs)), '\n')

for i, run in enumerate(run_dirs):
    print(i, run, '\n')

found 4 runs 

0 ../runs/gan_20190416_0011_gpu=1_bs=4_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=25_glw=[100_10_100_1]_clw=[1_1_10]_tag=brains_regular_gen/predict 

1 ../runs/gan_20190418_1124_gpu=4_bs=4_enc=[16_32_64_128]_dec=[128_64_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=25_glw=[100.0_10.0_5000.0_1.0]_clw=[1_1_10]_tag=brains_deep_high_prec_loss/predict 

2 ../runs/gan_20190418_1255_gpu=2_bs=4_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=25_glw=[100.0_10.0_5000.0_1.0]_clw=[1_1_10]_tag=brains_reg_gen_high_prec/predict 

3 ../runs/gan_20190419_0002_gpu=0_bs=4_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_vr=1.0_ti=False_is=5_cs=5_rf=10_rs=25_glw=[100.0_10.0_0.0_1.0]_clw=[1_1_10]_tag=brains_reg_gen_no_ti/predict 



In [66]:
# set run dir
run_dir = run_dirs[1]

pre_dirs = glob.glob(os.path.join(glob.escape(run_dir), '*', '*'))
pre_dirs = sorted(pre_dirs)

print('found {} pre_dirs'.format(len(pre_dirs)))

for i, pre_dir in enumerate(pre_dirs):
    #print(i, os.path.basename(pre_dir))
    print(i, pre_dir.split('/')[-2] + '/' + pre_dir.split('/')[-1])

found 8 pre_dirs
0 eval/gen_400
1 eval/gen_400_04
2 longterm/gen_400
3 test/gen_200_04
4 test/gen_400_04
5 test/gen_400_08
6 test/gen_550_04
7 test/gen_650_04


In [91]:
# generator config, run_dir, etc...
pre_dir = pre_dirs[4]

csv_path = os.path.join(pre_dir, 'meta.csv')
csv = pd.read_csv(csv_path)

print('found meta.csv: {}'.format(csv.shape))

found meta.csv: (271, 37)


In [106]:
# classifier
clf_dir = '../../../../backup/agemorph/models/clf/clf_20190330_1549_gpu=4_bs=8_lr=0.001_b1=0.9_b2=0.999_ep=0.1_bn=True_ds=1_lw=[1.0]/'
model_name = 'clf_100'
model_file = os.path.join(clf_dir, model_name+'.h5')

if os.path.isfile(model_file):
    print('found model file')

found model file


In [107]:
# gpu config
gpu_id = 7

if gpu_id is not None:                                                     
    gpu = '/gpu:' + str(gpu_id)                                            
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)                       
    config = tf.ConfigProto()                                              
    config.gpu_options.allow_growth = True                                 
    config.allow_soft_placement = True                                     
    set_session(tf.Session(config=config))                                 
else:                                                                      
    gpu = '/cpu:0'

In [108]:
print('starting predict')

with tf.device(gpu):
    
    # load classifier model
    classifier = keras.models.load_model(model_file)
    
    for i, (_, row) in enumerate(csv.iterrows()):
        
        if i % 10 == 0:
            print('step', i)
        
        xr = load_nii(row['img_path_0'])[None, ..., None]
        yr = load_nii(row['img_path_1'])[None, ..., None]
        yf = load_nii(row['img_path_yf'])[None, ..., None]
        
        batch = np.concatenate([xr, yr, yf], axis=0)
        
        pred = classifier.predict([batch])
        
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_xr'] = pred[0, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yr'] = pred[1, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yf'] = pred[2, 1]
        
print('finished predict')

csv_out_path = os.path.join(pre_dir, model_name+'.csv')

print('writing csv to {}'.format(csv_out_path))

csv.to_csv(csv_out_path, index=False)

starting predict
step 0
step 10
step 20
step 30
step 40
step 50
step 60
step 70
step 80
step 90
step 100
step 110
step 120
step 130
step 140
step 150
step 160
step 170
step 180
step 190
step 200
step 210
step 220
step 230
step 240
step 250
step 260
step 270
finished predict
writing csv to ../runs/gan_20190418_1124_gpu=4_bs=4_enc=[16_32_64_128]_dec=[128_64_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=25_glw=[100.0_10.0_5000.0_1.0]_clw=[1_1_10]_tag=brains_deep_high_prec_loss/predict/test/gen_650_04/clf_100.csv


In [92]:
clf_csvs = glob.glob(os.path.join(glob.escape(pre_dir), '[!meta]*.csv'))
clf_csvs = sorted(clf_csvs)

for i, clf_csv in enumerate(clf_csvs):
    print(i, os.path.basename(clf_csv))

0 clf_100.csv
1 clf_200.csv
2 reg_200.csv


In [94]:
csv_path = clf_csvs[0]
csv = pd.read_csv(csv_path)
print(csv.shape)

(271, 41)


In [95]:
def get_metric(csv_in, col, t, metric):

    true_pos = ((csv_in[col] > t) & (csv_in.pat_dx_1 == 3)).sum()
    true_neg = ((csv_in[col] <= t) & (csv_in.pat_dx_1 == 2)).sum()
    false_pos = ((csv_in[col] > t) & (csv_in.pat_dx_1 == 2)).sum()
    false_neg = ((csv_in[col] <= t) & (csv_in.pat_dx_1 == 3)).sum()
    
    true = true_pos + true_neg
    false = false_pos + false_neg

    if metric == 'f1':
        prec = true_pos / (true_pos + false_pos)
        reca = true_pos / (true_pos + false_neg)

        f1_score = 2 * (prec * reca) / (prec + reca)
        m = f1_score
        
    elif metric == 'acc':
        acc_p = true_pos / (true_pos + false_neg)
        acc_s = true_neg / (true_neg + false_pos)

        balanced_acc = (acc_p + acc_s) / 2
        m = balanced_acc
    
    return (round(m, 3), true, false)

In [96]:
def get_best_threshold(csv_in, col, metric):
    l = []
    
    for t in np.arange(0, 1, 0.01):
        m = get_metric(csv_in, col, t, metric)
        l.append((*m, t))
        
    return max(l)[3]

In [97]:
def get_split_metric(csv_in, col, cv_split, metric):
    
    csv_split = csv_in[csv_in.cv_split == cv_split]
    csv_other = csv_in[csv_in.cv_split != cv_split]
    
    t = get_best_threshold(csv_other, col, metric)
    
    m = get_metric(csv_split, col, t, metric)
    print('split {}: {}'.format(cv_split, m))
    
    return m

In [98]:
def get_cv_metric(csv_in, col, metric):
    l = [get_split_metric(csv_in, col, s, metric)[0] for s in range(5)]
    return np.array(l).mean()

In [ ]:
# create 5 folds
csv = csv.sample(frac=1).reset_index(drop=True)

pMCI = csv[csv.pat_dx_1 == 3].img_id_0.values
sMCI = csv[csv.pat_dx_1 == 2].img_id_0.values

p_bins = np.linspace(0, len(pMCI), 6).astype(int)[1:-1]
s_bins = np.linspace(0, len(sMCI), 6).astype(int)[1:-1]

p_splits = np.split(pMCI, p_bins)
s_splits = np.split(sMCI, s_bins)

In [ ]:
# write folds to csv
for i, (p_split, s_split) in enumerate(zip(p_splits, s_splits)):
    csv.loc[csv.img_id_0.isin(p_split), 'cv_split'] = i
    csv.loc[csv.img_id_0.isin(s_split), 'cv_split'] = i

In [ ]:
get_cv_metric(csv, 'pAD_xr', 'f1')

In [ ]:
get_cv_metric(csv, 'pAD_yr', 'f1')

In [ ]:
get_cv_metric(csv, 'pAD_yf', 'f1')

In [ ]:
get_cv_metric(csv, 'pAD_xr', 'acc')

In [ ]:
get_cv_metric(csv, 'pAD_yr', 'acc')

In [ ]:
get_cv_metric(csv, 'pAD_yf', 'acc')